# testing

## run tests


### test cells?


In [ ]:
const Mocha = require('mocha');

// call test runner
let mocha;
if (typeof mocha === 'undefined') {
    mocha = new Mocha({
        ui: 'bdd',
        reporter: 'list',
        reporterOptions: {
            stream: false,
        },
        //parallel: true,
        isWorker: true,
        jobs: 10,
        timeout: 10000,
    });
}

function resetTests(suite) {
    suite.tests.forEach(function (t) {
        delete t.state;
        t.timedOut = false;
    });
    suite.suites.forEach(resetTests);
    suite.tests = [];
}

function testCells(cells, ctx = {}) {
    resetTests(mocha.suite);
    if (!cells) {
        cells = 'mocha test the test cell runner';
    }
    if (!cells[0].code) {
        cells = typeof cells === 'string'
            //   multi-cell syntax           single cell syntax notebook.ipynb[cell ref]
            && (!cells.includes('.ipynb') || cells.includes('['))
            ? [importer.interpret(cells)]
            : importer.interpret(cells);
    }

    // TODO: add multi lingual support here

    cells.forEach(r => {
        mocha.suite.emit('pre-require', ctx, r.id, mocha);
        //Object.assign(ctx, Mocha);
        // TODO: add other require statements below to interent includes?
        try {
            console.log('testing cell:', r.id)
            var required = importer.import(r.id, ctx);
            Object.assign(ctx, required);
        } catch (e) {
            console.log(e);
        }
        mocha.suite.emit('require', required, r.id, mocha);
        mocha.suite.emit('post-require', ctx, r.id, mocha);
    });
    return new Promise(resolve => mocha.run(function (failures) {
        resolve(failures);
    }));
}

module.exports = testCells;



### mocha test this notebook

mocha test the test cell runner?


In [ ]:
const assert = require('assert');
const sinon = require('sinon');
const proxyquire = require('proxyquire');
const EventEmitter = require('events');

describe('testCells', function () {
  let testCells;
  let importerStub;
  let mochaRunStub;
  let suiteMock;
  let mochaMock;

  beforeEach(() => {
    suiteMock = new EventEmitter();
    suiteMock.tests = [];
    suiteMock.suites = [];
    suiteMock.emit = sinon.spy();

    mochaRunStub = sinon.stub().callsFake(cb => cb(0));

    mochaMock = function () {
      return {
        suite: suiteMock,
        run: mochaRunStub
      };
    };
    mochaMock.prototype = function () { };
    mochaMock.prototype.ui = sinon.stub();
    mochaMock.prototype.reporter = sinon.stub();
    mochaMock.prototype.timeout = sinon.stub();

    importerStub = {
      interpret: sinon.stub(),
      import: sinon.stub()
    };

    testCells = importer.import('test cells', {
      require: function (string) {
        return proxyquire(string, {
          'mocha': mochaMock,
        })
      },
      importer: importerStub,
      cached: false,
    });
  });

  it('should handle null cells input and interpret string', async () => {
    importerStub.interpret.returns({ id: 'test-id', code: 'describe("sample", () => it("passes", () => {}));' });
    importerStub.import.returns({});

    const failures = await testCells(null);
    assert.strictEqual(failures, 0);
    sinon.assert.called(importerStub.interpret);
    sinon.assert.called(importerStub.import);
  });

  it('should interpret a string path (non-ipynb)', async () => {
    importerStub.interpret.returns([{ id: 'test-id', code: 'describe("test", function () {})' }]);
    importerStub.import.returns({});

    const failures = await testCells('test.js');
    assert.strictEqual(failures, 0);
  });

  it('should wrap raw code string as an array if not array', async () => {
    importerStub.interpret.returns({ id: 'inline-code', code: 'describe("inline", function () {})' });
    importerStub.import.returns({});

    const failures = await testCells('describe("inline", function () {})');
    assert.strictEqual(failures, 0);
  });

  it('should interpret array of cell objects', async () => {
    const cells = [{ id: 'a', code: 'describe("suite", function () {})' }];
    importerStub.import.returns({});

    const failures = await testCells(cells);
    assert.strictEqual(failures, 0);
  });

  it('should handle import error gracefully', async () => {
    const cells = [{ id: 'b', code: 'describe("suite", function () {})' }];
    importerStub.import.throws(new Error('import error'));

    const failures = await testCells(cells);
    assert.strictEqual(failures, 0);
  });
});


## test runner?

watch files run tests?

How to use: 

```
node -e "require('./Core').import('watch files run tests')('**/zuora*.ipynb', 'zuora to eloqua.ipynb')"

node -e "require('./Core').import('watch files run tests')('**/*.ipynb', 'test test runner')"```

TODO: restart this test script every loop? fork and abandon current thread?

TODO: update cell cache in intrepret notebooks.ipynb

TODO: git apply without whitespace, reset the rest?  separate index?  how do make git-scenario app?

TODO: re-import cells on Utilities/.modules folder change. run all "watcher" commands from a test below?  mocha.grep?





In [ ]:
var chokidar = require("chokidar");
var importer = require('../Core');
var testCells = importer.import('test cells');

// TODO: code analysis to combine blocks into modules?

var rateLimiter, done = true;
function testWatcher(files, tests) {
    files = typeof files === 'string' ? [files] : files;
    console.log('watching ' + files + ' - ' + path.resolve('.'))
    var watcher = chokidar.watch(files, {
        interval: 1000,
        atomic: 1000,
        awaitWriteFinish: true
    });
    watcher.on("change", function (event, path) {
        if (!done) {
            return;
        }
        console.log('running all tests');
        done = false;
        return testCells(tests).then(() => (done = true))
    });
    testCells(tests);

    var stdin = process.openStdin();
    stdin.addListener("data", function (d) {
        stdin.close();
    });
}
module.exports = testWatcher;



## web service



test sigma index?

ROUTE[] = /sigma

ROUTE[] = /sigma/:project


In [ ]:
const path = require('path')
const fs = require('fs')
const Mustache = require('mustache')
const { listInProject } = importer.import('list project files');

const PROFILE_HOME = path.resolve(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE)
const GITHUB_BARE = path.join(PROFILE_HOME, 'git-bare')

function listNotebooks() {
  let notebooks = listInProject(path.resolve(__dirname, '../'), '{,*,*/*,*/*/*,*/*/*/*}*.ipynb')
  return notebooks
    .map(nb => path.relative(path.resolve(__dirname, '../'), nb).split(path.sep)[0])
    .filter((a, i, arr) => !a.endsWith('.ipynb') && arr.indexOf(a) === i)
}

async function sigmaIndex(project) {
  if (project && !fs.existsSync(path.join(GITHUB_BARE, project))) {
    throw new Error('Project not found')
  }

  let SERVICES = []

  if (!project) {
    const ORIGINAL_COLORS = importer.import('notebook-colors.js')
    SERVICES = listNotebooks().map(NAME => ({
      NAME,
      COLOR: ORIGINAL_COLORS[NAME]
    }))
  } else {
    const { generateHues, hslToRgb, rgbToInt } = importer.import('color-fader.js')
    const serveServices = importer.import('git service list')
    const servicePaths = await serveServices(project)
    const hues = generateHues(servicePaths.length)

    SERVICES = servicePaths.map((NAME, i) => {
      const rgb = hslToRgb(hues[i], 70, 50)
      const hex = rgbToInt(...rgb).toString(16).padStart(6, '0')
      return { NAME, COLOR: `#${hex}` }
    })
  }

  const TEMPLATE = importer.interpret('test sigma template').code
  const result = Mustache.render(TEMPLATE, {
    SERVICES,
    GRAPH_URL: !project ? '/node-graph.json' : `/${project}/node-graph.json`,
    TIMESTAMP: Date.now()
  })

  return result
}

module.exports = sigmaIndex



## frontend



### index page

test sigma template?


In [ ]:
<!DOCTYPE html>
<html lang="en">

<head>
  <meta charset="UTF-8">
  <title>Code Doily</title>
  <!-- Load Sigma.js v2 -->
  <link href="/sigma-style.css" rel="stylesheet" />
  <script src="https://cdnjs.cloudflare.com/ajax/libs/graphology/0.26.0/graphology.umd.min.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/sigma@2.3.0/build/sigma.min.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/graphology-library/dist/graphology-library.min.js"></script>
</head>

<body>

  <div id="sigma-container"></div>
  <div id="info">Zoom & drag to explore</div>
  <div id="legend">
    <div class="legend-item">
      <div class="legend-color" style="background-color: #6699cc;"></div> Imports
    </div>
    <div class="legend-item">
      <div class="legend-color" style="background-color: #ff9966;"></div> Dependencies
    </div>
    {{#SERVICES}}
    <div class="legend-item">
      <div class="legend-color" style="background-color: {{COLOR}};"></div> {{NAME}}
    </div>
    {{/SERVICES}}
  </div>
  <script>const GRAPH = '{{{GRAPH_URL}}}?t={{TIMESTAMP}}';</script>
  <script src="/animate-nodes.js?t={{TIMESTAMP}}"></script>
  <script src="/render-remote.js?t={{TIMESTAMP}}"></script>
  <script src="/color-fader.js?t={{TIMESTAMP}}"></script>
  <script src="/update-view.js?t={{TIMESTAMP}}"></script>
  <script src="/highlight-nodes.js?t={{TIMESTAMP}}"></script>
  <script src="/notebook-colors.js?t={{TIMESTAMP}}"></script>
  <script src="/trace-nodes.js?t={{TIMESTAMP}}"></script>
  <script src="/sigma-script.js?t={{TIMESTAMP}}"></script>
</body>

</html>


### service colors

notebook-colors.js?

ROUTE = /notebook-colors.js


In [ ]:

const originalColors = {
  Algorithms: '#9b59b6',   // purple
  Analytics: '#27ae60',    // green
  Art: '#e91e63',          // pink
  "Cloud Services": '#f1c40f', // yellow
  Core: '#7f8c8d',         // gray
  Databases: '#1abc9c',    // teal
  Docker: '#c0392b',       // red
  Exercises: '#8e44ad',    // dark purple
  Frameworks: '#2ecc71',   // light green
  Frontends: '#d35400',    // burnt orange (still distinct from pure orange)
  Games: '#34495e',        // dark slate
  Google: '#95a5a6',       // light gray
  Languages: '#16a085',    // turquoise
  Marketing: '#f39c12',    // gold
  Selenium: '#bdc3c7',     // silver
  Utilities: '#e67e22'     // carrot (dark orange, distinguishable)
};

if (typeof module != 'undefined') {
  module.exports = originalColors
}



### stylesheet

sigma-style.css?

ROUTE = /sigma-style.css


In [ ]:
html,
body {
  margin: 0;
  height: 100%;
  overflow: hidden;
  font-family: sans-serif;
}

#sigma-container {
  width: 100vw;
  height: 100vh;
}

#info {
  position: absolute;
  top: 10px;
  left: 10px;
  background: rgba(255, 255, 255, 0.9);
  padding: 10px;
  border-radius: 6px;
  box-shadow: 0 0 5px rgba(0, 0, 0, 0.2);
  z-index: 100;
}

#legend {
  position: absolute;
  bottom: 20px;
  left: 20px;
  background: rgba(255, 255, 255, 0.9);
  border: 1px solid #ccc;
  border-radius: 8px;
  padding: 10px;
  font-family: sans-serif;
  font-size: 14px;
}

.legend-item {
  display: flex;
  align-items: center;
  margin-bottom: 5px;
}

.legend-color {
  width: 16px;
  height: 16px;
  margin-right: 8px;
  border-radius: 3px;
}

### init graph

sigma-script.js?

ROUTE = /sigma-script.js


In [ ]:
const graph = new graphology.Graph();
const container = document.getElementById('sigma-container');
const renderer = new Sigma(graph, container, {
  renderEdgeLabels: false,
  allowInvalidContainer: false,
  settings: { zIndex: true },
});

renderer.on('clickStage', clearSelection);
renderer.on('clickNode', highlightNodes.bind(null, renderer));
container.addEventListener('dblclick', handleDoubleClick);
renderer.getCamera().on('updated', updateView.bind(null, renderer, null));
initRemote();


// ----------- FUNCTIONS -----------

function getServiceColorsFromLegend() {
  const colors = {};
  document.querySelectorAll('.legend-item').forEach(el => {
    colors[el.innerText.trim()] = el.children[0].style.backgroundColor;
  });
  return Object.keys(colors).length > 0 ? colors : originalColors;
}

function getServices(serviceColors) {
  return Object.keys(serviceColors || {}).length
    ? Object.keys(serviceColors)
    : [
      'Auth Service', 'User DB', 'Frontend', 'Billing Service',
      'Notification', 'Search Engine', 'Cache', 'API Gateway',
      'Logging', 'Monitoring', 'AI Recommender', 'Backup Service'
    ];
}

function clearSelection() {
  lastClickedNode = null;
  graph.forEachNode((n, attributes) => {
    graph.setNodeAttribute(n, 'isPrimary', false);
    graph.setNodeAttribute(n, 'isSelected', false);
    graph.setNodeAttribute(n, 'isSource', false);
    graph.setNodeAttribute(n, 'isTarget', false);
    graph.setNodeAttribute(n, 'zIndex', 1);
    graph.setNodeAttribute(n, 'size', 10);
  });
  graph.forEachEdge(e => {
    graph.setEdgeAttribute(e, 'zIndex', 1);
    graph.setEdgeAttribute(e, 'color', '#ccc');
  });
  updateView(renderer);
}

function handleDoubleClick() {
  renderer.getCamera().animatedReset();
  if (!lastClickedNode) return;

  const attributes = graph.getNodeAttributes(lastClickedNode);
  const notebookFile = attributes.id.replace(/.ipynb.*/, '');
  const docsCell = parseInt((/.ipynb\[([0-9]+)\]/gi).exec(attributes.id)?.[1]);
  const basePath = `${window.location.origin}/docs/${attributes.service}/${safeurl(notebookFile)}`;
  const fullPath = isNaN(docsCell)
    ? `${basePath}/index.html`
    : `${basePath}/${safeurl(attributes.cell || ('cell_' + docsCell))}.html`;

  window.open(fullPath, '_docs');
}

function safeurl(url) {
  return url.replace(/[^a-z0-9_-]/ig, '_').substr(0, 100);
}

let serviceColors = getServiceColorsFromLegend();
function updateLegend(services) {
  const legendContainer = document.getElementById('legend');
  if (!legendContainer) return [];
  while (legendContainer.children.length > 2) {
    legendContainer.removeChild(legendContainer.lastChild);
  }
  //legendContainer.innerHTML = '';
  const hues = generateHues(services.length)
  const renderedServices = services.reduce((obj, service, i) => {
    const item = document.createElement('div');
    item.className = 'legend-item';

    const colorBox = document.createElement('div');
    colorBox.className = 'legend-color';
    const rgb = hslToRgb(hues[i], 70, 50)
    const hex = rgbToInt(...rgb).toString(16).padStart(6, '0')
    colorBox.style.backgroundColor = '#' + hex;

    item.appendChild(colorBox);
    item.appendChild(document.createTextNode(service));
    legendContainer.appendChild(item);

    obj[service] = colorBox.style.backgroundColor
    return obj
  }, {});

  return renderedServices;
}

async function initRemote() {
  const response = await fetch(typeof GRAPH !== 'undefined' ? GRAPH : '/node-graph.json');
  const data = await response.json();
  serviceColors = updateLegend(data.services)
  renderRemote(renderer, data);
}

// Optional: demo graph rendering
const services = getServices(serviceColors);

renderRemote(renderer, {
  nodes: services.map(service => ({ id: service, label: service, service })),
  edges: Array.from({ length: services.length * 1.5 }, () => ({
    source: services[Math.floor(Math.random() * services.length)],
    target: services[Math.floor(Math.random() * services.length)]
  }))
});


### trace nodes?

show a flash on the map when a page from another service loads

ROUTE = /trace-nodes.js

In [ ]:

let NOTIFIED = {}

function handleTraces(message) {
  message.notebook.forEach(notebook => {
    if (NOTIFIED[notebook] + 2000 > Date.now()) return;
    NOTIFIED[notebook] = Date.now();
    graph.forEachNode((nodeId, attributes) => {
      if (nodeId.startsWith(notebook + '.ipynb')) {
        if (!attributes.touched || attributes.touched + 2000 <= Date.now()) {
          graph.setNodeAttribute(nodeId, 'touched', Date.now());
          colorChanged = true;
        }
      }
    });
  });
}

if (typeof TRACES != 'undefined') {
  TRACES.push(handleTraces);
}



### mouse click

highlight-nodes.js?

ROUTE = /highlight-nodes.js


In [ ]:
let lastClickTime = 0;
const doubleClickThreshold = 300; // ms

function highlightNodes(renderer, { data, node }) {
  const graph = renderer.getGraph();
  const nodeId = node;
  const now = Date.now();

  lastClickTime = now;
  lastClickedNode = node;

  resetAllGraphStyles(graph);
  markPrimaryNode(graph, nodeId);
  recursivelyHighlightConnections(graph, nodeId);

  sortGraphByZIndex(graph);
  updateView(renderer);
  renderer.refresh();
}

// ---------- Helper Functions ------------

function resetAllGraphStyles(graph) {
  graph.forEachNode((n) => {
    graph.setNodeAttribute(n, 'isPrimary', false);
    graph.setNodeAttribute(n, 'isSelected', false);
    graph.setNodeAttribute(n, 'isTarget', false);
    graph.setNodeAttribute(n, 'isSource', false);
    graph.setNodeAttribute(n, 'zIndex', 1);
    graph.setNodeAttribute(n, 'size', 10);
  });

  graph.forEachEdge((e) => {
    graph.setEdgeAttribute(e, 'zIndex', 1);
    graph.setEdgeAttribute(e, 'color', '#ccc');
  });
}

function markPrimaryNode(graph, nodeId) {
  graph.setNodeAttribute(nodeId, 'isPrimary', true);
  graph.setNodeAttribute(nodeId, 'isSelected', true);
  graph.setNodeAttribute(nodeId, 'size', 11);
  graph.setNodeAttribute(nodeId, 'zIndex', 9999);
}

function recursivelyHighlightConnections(graph, rootId) {
  const visited = new Set();

  function traverse(currentId, isInbound, isOutbound) {
    if (visited.has(currentId)) return;
    visited.add(currentId);

    if (isOutbound) {
      graph.forEachOutboundEdge(currentId, (edge, _, __, target) => {
        if (!visited.has(target)) {
          markDirectionalNode(graph, edge, target, 'isTarget', '#6699cc');
          traverse(target, false, true);
        }
      });
    }

    if (isInbound) {
      graph.forEachInboundEdge(currentId, (edge, _, source) => {
        if (!visited.has(source)) {
          markDirectionalNode(graph, edge, source, 'isSource', '#ff9966');
          traverse(source, true, false);
        }
      });
    }

    if (!isOutbound) {
      graph.forEachOutboundEdge(currentId, (edge, _, __, target) => {
        if (!visited.has(target)) {
          markLinkedNode(graph, edge, target);
        }
      });
    }

    if (!isInbound) {
      graph.forEachInboundEdge(currentId, (edge, _, source) => {
        if (!visited.has(source)) {
          markLinkedNode(graph, edge, source);
        }
      });
    }
  }

  traverse(rootId, true, true);
}

function markDirectionalNode(graph, edge, nodeId, role, color, z = 8888) {
  graph.setNodeAttribute(nodeId, 'isSelected', true);
  graph.setNodeAttribute(nodeId, role, true); // 'isSource' or 'isTarget'
  graph.setNodeAttribute(nodeId, 'zIndex', z);
  graph.setEdgeAttribute(edge, 'zIndex', z);
  graph.setEdgeAttribute(edge, 'color', color);
}

function markLinkedNode(graph, edge, nodeId) {
  graph.setNodeAttribute(nodeId, 'isSelected', true);
  graph.setNodeAttribute(nodeId, 'zIndex', 7777);
  graph.setEdgeAttribute(edge, 'zIndex', 7777);
}

function sortGraphByZIndex(graph) {
  const sortMap = (map) =>
    new Map([...map.entries()].sort((a, b) =>
      (a[1].attributes.zIndex ?? 1) - (b[1].attributes.zIndex ?? 1)));

  graph._nodes = sortMap(graph._nodes);
  graph._edges = sortMap(graph._edges);
}



### color fader

color-fader.js?

ROUTE = /color-fader.js

In [ ]:
function generateHues(count) {
  const colors = [];
  const step = 360 / count;

  for (let i = 0; i < count; i++) {
    const hue = Math.round((i * step) % 360);
    colors.push(hue);
  }

  return colors;
}

function hexToRgb(hex) {
  var result = /^#?([a-f\d]{2})([a-f\d]{2})([a-f\d]{2})$/i.exec(hex);
  if (!result) {
    result = /^#?([a-f\d])([a-f\d])([a-f\d])$/i.exec(hex);
  }
  return result ? {
    r: parseInt(result[1].length == 1 ? (result[1] + result[1]) : result[1], 16),
    g: parseInt(result[2].length == 1 ? (result[2] + result[2]) : result[2], 16),
    b: parseInt(result[3].length == 1 ? (result[3] + result[3]) : result[3], 16)
  } : null;
}

function intToRgb(int) {
  const r = (int >> 16) & 0xFF;
  const g = (int >> 8) & 0xFF;
  const b = int & 0xFF;
  return { r, g, b };
}

function rgbToInt(r, g, b) {
  return (r << 16) + (g << 8) + b;
}

function rgbToHsl(r, g, b) {
  r /= 255;
  g /= 255;
  b /= 255;
  const max = Math.max(r, g, b), min = Math.min(r, g, b);
  let h, s, l = (max + min) / 2;

  if (max === min) {
    h = s = 0; // achromatic
  } else {
    const d = max - min;
    s = l > 0.5 ? d / (2 - max - min) : d / (max + min);
    switch (max) {
      case r:
        h = ((g - b) / d + (g < b ? 6 : 0)) * 60;
        break;
      case g:
        h = ((b - r) / d + 2) * 60;
        break;
      case b:
        h = ((r - g) / d + 4) * 60;
        break;
    }
  }

  return [h, s * 100, l * 100];
}

function hslToRgb(h, s, l) {
  s /= 100;
  l /= 100;

  const c = (1 - Math.abs(2 * l - 1)) * s;
  const x = c * (1 - Math.abs((h / 60) % 2 - 1));
  const m = l - c / 2;

  let r = 0, g = 0, b = 0;

  if (0 <= h && h < 60) [r, g, b] = [c, x, 0];
  else if (60 <= h && h < 120) [r, g, b] = [x, c, 0];
  else if (120 <= h && h < 180) [r, g, b] = [0, c, x];
  else if (180 <= h && h < 240) [r, g, b] = [0, x, c];
  else if (240 <= h && h < 300) [r, g, b] = [x, 0, c];
  else if (300 <= h && h < 360) [r, g, b] = [c, 0, x];

  return [
    Math.round((r + m) * 255),
    Math.round((g + m) * 255),
    Math.round((b + m) * 255)
  ];
}

function interpolateHue(h1, h2, percent) {
  let diff = h2 - h1;
  if (diff > 180) diff -= 360;
  else if (diff < -180) diff += 360;
  return (h1 + diff * percent + 360) % 360; // add 360 to avoid negative hue wrap
}

function interpolateHSL(hsl1, hsl2, percent) {
  const [h1, s1, l1] = hsl1;
  const [h2, s2, l2] = hsl2;
  const h = interpolateHue(h1, h2, percent);
  const s = s1 + (s2 - s1) * percent;
  const l = l1 + (l2 - l1) * percent;
  return [h, s, l];
}

if (typeof module != 'undefined') {
  module.exports = {
    generateHues,
    hslToRgb,
    rgbToInt,
  }
}



### animate graph

update-view.js?

ROUTE = /update-view.js


In [ ]:
// Refactored and modularized graph animation code

let hasSelected = false;
let cancelAnimation, cancelAnimation2;
let colorChanged = false;
let animator, targeter;
let layout, subGraph;
let changed = false;

function updateVisibility(renderer, graph) {
  hasSelected = false;
  graph.forEachNode((nodeKey, attributes) => {
    const pos = renderer.graphToViewport(attributes);
    const visible = attributes.isSelected || (
      pos.x >= 0 && pos.x <= window.innerWidth &&
      pos.y >= 0 && pos.y <= window.innerHeight
    );
    graph.setNodeAttribute(nodeKey, 'isVisible', visible);
    if (attributes.isSelected) hasSelected = true;
  });
}

function connectNodeEdges(graph, nodeKey, visibleIds, visibleEdges) {
  const addEdge = (edge, attrs, source, target) => {
    if (!visibleIds[source]) {
      visibleIds[source] = true;
      subGraph.addNode(source, { ...graph.getNodeAttributes(source), isIncluded: true });
    }
    if (!visibleIds[target]) {
      visibleIds[target] = true;
      subGraph.addNode(target, { ...graph.getNodeAttributes(target), isIncluded: true });
    }
    if (!visibleEdges[source + target]) {
      visibleEdges[source + target] = true;
      subGraph.addEdge(source, target, { ...attrs });
    }
  };
  graph.forEachInboundEdge(nodeKey, addEdge);
  graph.forEachOutboundEdge(nodeKey, addEdge);
}

function updateView(renderer, originalLayout) {
  const graph = renderer.getGraph();
  subGraph = subGraph || new graphology.Graph();
  subGraph.clear();

  updateVisibility(renderer, graph);
  if (originalLayout) layout = originalLayout;

  const visibleIds = {}, visibleEdges = {};
  graph.forEachNode((nodeKey, attr) => {
    if (!visibleIds[nodeKey]) {
      visibleIds[nodeKey] = true;
      subGraph.addNode(nodeKey, { ...attr, isIncluded: true });
    }
    if (!hasSelected || !attr.isVisible || attr.isSelected) {
      connectNodeEdges(graph, nodeKey, visibleIds, visibleEdges);
    }
  });

  colorChanged = true;
  changed = true;

  startColorAnimator(renderer);
}

function startColorAnimator(renderer) {
  if (!animator) animator = setInterval(() => reGraph(renderer), 100);
}

function startPositionAnimator(renderer) {
  if (!targeter) targeter = setInterval(() => reTarget(renderer), 200);
}

function getNodeColor(attr) {
  const now = Date.now();
  if (attr.touched + 500 > now) return '#fff';
  if (attr.touched + 1000 > now) return '#000';
  if (attr.touched + 1500 > now) return '#fff';

  if (!hasSelected) return serviceColors[attr.service];
  if (attr.isPrimary) return '#000';
  if (attr.isTarget) return '#6699cc';
  if (attr.isSource) return '#ff9966';
  return '#999';
}

function projectNodePosition(nodeKey, attr) {
  if (!hasSelected || !layout[nodeKey]) return;
  if (attr.isSelected) return;
  const { x, y } = layout[nodeKey];
  const angle = Math.atan2(y, x);
  const distance = 1000;
  subGraph.setNodeAttribute(nodeKey, 'x', Math.cos(angle) * distance);
  subGraph.setNodeAttribute(nodeKey, 'y', Math.sin(angle) * distance);
}

function reGraph(renderer) {
  const state = renderer.getCamera().getState();

  if (changed || hasSelected) {
    subGraph.forEachNode((nodeKey, attr) => projectNodePosition(nodeKey, attr));
    changed = false;
  }

  if (colorChanged) {
    const graph = renderer.getGraph();
    graph.forEachNode((nodeKey, attr) => {
      subGraph.setNodeAttribute(nodeKey, 'color', getNodeColor(attr));
    });
  }

  const settings = graphologyLibrary.layoutForceAtlas2.inferSettings(subGraph);
  graphologyLibrary.layoutForceAtlas2.assign(subGraph, {
    iterations: 2,
    settings: {
      ...settings,
      gravity: hasSelected ? 0.2 : 2,
      strongGravityMode: true,
      edgeWeightInfluence: 0.1,
      linLogMode: false
    }
  });

  startPositionAnimator(renderer);
}

function reTarget(renderer) {
  const graph = renderer.getGraph();
  const speed = 100;
  const targetPositions = {}, targetColors = {};

  subGraph.forEachNode((nodeKey, { x: tx, y: ty, color }) => {
    if (!graph.hasNode(nodeKey)) return;
    const { x, y } = graph.getNodeAttributes(nodeKey);
    const dx = tx - x, dy = ty - y;
    const distSq = dx * dx + dy * dy;
    if (distSq === 0) return;
    const scale = Math.min(speed, Math.sqrt(distSq)) / Math.sqrt(distSq);

    targetPositions[nodeKey] = { x: x + dx * scale, y: y + dy * scale };
    targetColors[nodeKey] = { color };
  });

  cancelAnimation = animateNodes(graph, targetPositions, {
    duration: 200,
    easing: 'quadraticInOut'
  });
  cancelAnimation2 = animateNodes(graph, targetColors, {
    duration: 400,
    easing: 'quadraticInOut'
  });
}



### load remote data

render-remote.js?

ROUTE = /render-remote.js


In [ ]:
let iterationCount = 0;
let forceInterval, sensibleSettings;

function forceGraph(graph, onFinish) {
  if (iterationCount++ === 200) {
    clearInterval(forceInterval);

    const originalLayout = Object.fromEntries(
      graph.mapNodes((key, attrs) => [key, { x: attrs.x, y: attrs.y }])
    );

    onFinish(originalLayout);
    return;
  }

  graphologyLibrary.layoutForceAtlas2.assign(graph, {
    iterations: 2,
    settings: {
      ...sensibleSettings,
      gravity: 2,
      strongGravityMode: true,
      linLogMode: false,
      edgeWeightInfluence: 0.1,
    },
  });
}

function resetGraph(graph, nodes, edges, onFinish) {
  graph.clear();

  const len = nodes.length;
  nodes.forEach((node, i) => {
    graph.addNode(node.id, {
      ...node,
      label: node.cell || node.route || node.id,
      x: 1000 * Math.cos((2 * Math.PI * i) / len),
      y: 1000 * Math.sin((2 * Math.PI * i) / len),
      size: 10,
      service: node.service,
      zIndex: 1,
    });
  });

  edges.forEach(({ source, target }) => {
    if (source !== target && !graph.hasEdge(source, target)) {
      graph.addEdge(source, target, {
        color: '#ccc',
        size: 4,
        zIndex: 1,
      });
    }
  });

  iterationCount = 0;
  sensibleSettings = graphologyLibrary.layoutForceAtlas2.inferSettings(graph);
  forceInterval = setInterval(() => forceGraph(graph, onFinish), 10);
}

function renderRemote(renderer, { nodes, edges }) {
  const graph = renderer.getGraph();

  cancelAnimation?.();
  cancelAnimation2?.();
  clearInterval(targeter);
  clearInterval(animator);
  clearInterval(forceInterval);

  targeter = animator = forceInterval = undefined;

  setTimeout(() => {
    resetGraph(graph, nodes, edges, (originalLayout) => {
      updateView(renderer, originalLayout);
      renderer.refresh();
    });
  }, 400);
}



### sigma animations

animate-nodes.js?

ROUTE = /animate-nodes.js


In [ ]:

const ANIMATE_DEFAULTS = {
  easing: "quadraticInOut",
  duration: 150,
};

easings_1 = {}
easings_1.cubicInOut = easings_1.cubicOut = easings_1.cubicIn = easings_1.quadraticInOut = easings_1.quadraticOut = easings_1.quadraticIn = easings_1.linear = void 0;
/**
 * Sigma.js Easings
 * =================
 *
 * Handy collection of easing functions.
 * @module
 */
var linear = function (k) { return k; };
easings_1.linear = linear;
var quadraticIn = function (k) { return k * k; };
easings_1.quadraticIn = quadraticIn;
var quadraticOut = function (k) { return k * (2 - k); };
easings_1.quadraticOut = quadraticOut;
var quadraticInOut = function (k) {
  if ((k *= 2) < 1)
    return 0.5 * k * k;
  return -0.5 * (--k * (k - 2) - 1);
};
easings_1.quadraticInOut = quadraticInOut;
var cubicIn = function (k) { return k * k * k; };
easings_1.cubicIn = cubicIn;
var cubicOut = function (k) { return --k * k * k + 1; };
easings_1.cubicOut = cubicOut;
var cubicInOut = function (k) {
  if ((k *= 2) < 1)
    return 0.5 * k * k * k;
  return 0.5 * ((k -= 2) * k * k + 2);
};
easings_1.cubicInOut = cubicInOut;
var easings = {
  linear: easings_1.linear,
  quadraticIn: easings_1.quadraticIn,
  quadraticOut: easings_1.quadraticOut,
  quadraticInOut: easings_1.quadraticInOut,
  cubicIn: easings_1.cubicIn,
  cubicOut: easings_1.cubicOut,
  cubicInOut: easings_1.cubicInOut,
};
easings_1.default = easings;


const index_1 = {}
index_1.requestFrame = typeof requestAnimationFrame !== "undefined"
  ? function (callback) { return requestAnimationFrame(callback); }
  : function (callback) { return setTimeout(callback, 0); };
index_1.cancelFrame = typeof cancelAnimationFrame !== "undefined"
  ? function (requestID) { return cancelAnimationFrame(requestID); }
  : function (requestID) { return clearTimeout(requestID); };


/**
 * Function used to animate the nodes.
 */
function animateNodes(graph, targets, opts, callback) {
  var options = Object.assign({}, ANIMATE_DEFAULTS, opts);
  var easing = typeof options.easing === "function" ? options.easing : easings_1.default[options.easing];
  var start = Date.now();
  var startPositions = {};
  for (var node in targets) {
    var attrs = targets[node];
    startPositions[node] = {};
    for (var k in attrs)
      startPositions[node][k] = graph.getNodeAttribute(node, k);
  }
  var frame = null;
  var step = function () {
    var p = (Date.now() - start) / options.duration;
    if (p >= 1) {
      // Animation is done
      for (var node in targets) {
        var attrs = targets[node];
        for (var k in attrs)
          graph.setNodeAttribute(node, k, attrs[k]);
      }
      if (typeof callback === "function")
        callback();
      return;
    }
    p = easing(p);
    for (var node in targets) {
      var attrs = targets[node];
      var s = startPositions[node];
      for (var k in attrs) {
        if (k === 'color') {
          let hex1 = hexToRgb(s[k])
          let hex2 = hexToRgb(attrs[k])
          if (hex1 && hex2) {
            let { r, g, b } = hex1
            let { r: r2, g: g2, b: b2 } = hex2
            const hsl1 = rgbToHsl(r, g, b);
            const hsl2 = rgbToHsl(r2, g2, b2);
            const interpolatedHSL = interpolateHSL(hsl1, hsl2, p);
            const interpolatedRGB = hslToRgb(...interpolatedHSL);
            const rgbInt = rgbToInt(...interpolatedRGB)
            graph.setNodeAttribute(node, k, '#' + rgbInt.toString(16))
          }

        } else {
          graph.setNodeAttribute(node, k, attrs[k] * p + s[k] * (1 - p));
        }
      }
    }
    frame = index_1.requestFrame(step);
  };
  step();
  return function () {
    if (frame)
      index_1.cancelFrame(frame);
  };
}



## backend



### graph JSON service

cell nodes?

ROUTE = /node-graph.json


In [ ]:
const fs = require('fs')
const path = require('path')
const { cacheCells } = importer.import('cache notebook')
const { cellCache } = importer.import('cell cache')

const NOTEBOOK_PATH = path.join(__dirname, '..')
const PROJECT_PATH = path.join(NOTEBOOK_PATH, '.notebook-graph.json')
let INTERPRET = {}, TIMES = {}

if (fs.existsSync(PROJECT_PATH)) {
  let { searches = {}, times = {} } = JSON.parse(fs.readFileSync(PROJECT_PATH))
  INTERPRET = searches
  TIMES = times
}

const IMPORT_REGEX = /(import_notebook|importNotebook|import|interpret)\([\\\n\r\[\]\s'"]*([\s\S]*?)[\\\n\r\]\[\s'"]*\)/gi
const clean = str => str.split(/[\\\n\r\s'"]*,[\\\n\r\s'"]*/gi).filter(Boolean)

const resolveSearch = s => {
  if (INTERPRET[s] === '') return []
  if (typeof INTERPRET[s] === 'object') return INTERPRET[s]
  try {
    INTERPRET[s] = importer.interpret(INTERPRET[s] || s)
  } catch (e) {
    INTERPRET[s] = e.message.includes('Nothing found') ? '' : (() => { throw e })()
  }
  const r = INTERPRET[s]
  return !r || typeof r === 'string' ? [] : Array.isArray(r) ? r : [r]
}

const getImportEdges = (...args) => clean(args[2]).flatMap(resolveSearch)

const getPath = val =>
  typeof val === 'string' ? val :
    Array.isArray(val) ? (val.length < 1 ? '' :
      val.length === 1 ? path.join(path.dirname(val[0].filename), val[0].id) :
        val[0].filename) :
      val.filename ? path.join(path.dirname(val.filename), val.id) : ''

const toNode = c => ({
  id: c.id,
  cell: ((c.questions || [])[0]) || '',
  route: (c.markdown.match(/ROUTE.*/) || [])[0] || '',
  service: path.relative(path.join(__dirname, '..'), c.filename).split(/[\\/]/)[0]
})

const saveGraph = (nodes, edges) => {
  const graph = {
    nodes: nodes.map(toNode),
    edges: edges.filter((e, i, a) => a.findIndex(x => x.source + x.target === e.source + e.target) === i),
    searches: Object.fromEntries(Object.entries(INTERPRET).map(([k, v]) => [k, getPath(v)])),
  }
  const services = [...new Set(graph.nodes.map(n => n.service))];
  graph.services = services
  fs.writeFileSync(PROJECT_PATH, JSON.stringify(graph, null, 4))
  return graph
}

const isValid = c => (((c.questions || [])[0] || '').length || 0) > 0 || c.markdown.includes('ROUTE')
const add = (cell, map, list) => map[cell.id] || (map[cell.id] = cell, list.push(cell))
const skip = f => f.includes('cache')

const buildGraph = (all, nodes = [], edges = [], seen = {}) => {
  for (const c of all) {
    if (skip(c.filename) || !isValid(c)) continue
    add(c, seen, nodes)
    for (const m of c.code.matchAll(IMPORT_REGEX)) {
      for (const t of getImportEdges(...m)) {
        if (skip(t.filename)) continue
        add(t, seen, nodes)
        edges.push({ source: c.id, target: t.id })
      }
    }
  }
  return { nodes, edges }
}

function serveNodes() {
  const all = cellCache.map(([_, cell]) => importer.lookupCell(cell, cacheCells))
  const { nodes, edges } = buildGraph(all)
  const graph = saveGraph(nodes, edges)
  return { nodes: graph.nodes, edges: graph.edges, services: graph.services.sort() }
}

module.exports = serveNodes



### repo JSON service

rewrite of the above function to work on any repo

git repo nodes?

ROUTE = /:project/node-graph.json


In [ ]:
const fs = require('fs');
const path = require('path');
const glob = require('glob');
const { safeurl } = importer.import('domain cache tools');
const getLanguage = importer.import('match language to grammar');

const PROFILE_HOME = path.resolve(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE);
const GITHUB_BARE = path.join(PROFILE_HOME, 'git-bare');
const CPP_REGEX = /^\s*#include\s*([<"])([^">]+)[">]/gm;

const extractCPPImports = (root, filepath) => {
  const content = fs.readFileSync(filepath, 'utf-8').toString();
  const baseDir = path.dirname(filepath);

  const matches = [...content.matchAll(CPP_REGEX)];
  return matches
    .filter(m => m[1] === '"') // Only include "quoted" (relative) includes
    .map(m => ({ relative: path.relative(root, path.resolve(baseDir, m[2])) }))
    .concat(matches
      .filter(m => m[1] === '<')
      .map(m => ({ library: m[2] })))
}

const toNode = (filepath, root) => ({
  id: path.basename(filepath),
  label: path.basename(filepath),
  service: path.dirname(filepath) || 'library',
})

const extractEdges = (root, lang, filepath) =>
  lang === 'cpp' || lang === 'C' ? extractCPPImports(root, filepath) : [];

const createNodeIfNeeded = (filepath, root, seen, nodes) => {
  if (seen[path.basename(filepath)]) return seen[filepath];
  const node = toNode(filepath, root);
  seen[path.basename(filepath)] = node;
  nodes.push(node);
  return node;
};

const processFile = async (filepath, root, seen, nodes, edges) => {
  const lang = await getLanguage(filepath);
  if (!lang) return;

  createNodeIfNeeded(path.relative(root, filepath), root, seen, nodes);

  const imports = extractEdges(root, lang, filepath);
  imports.forEach(({ relative, library }) => {
    createNodeIfNeeded(relative ? relative : library, relative ? root : '', seen, nodes);
    edges.push({
      source: path.basename(filepath),
      target: path.basename(relative ? relative : library),
    });
  });
};

async function serveNodes(project) {
  const root = path.join(GITHUB_BARE, project);
  if (!project || !fs.existsSync(root)) throw new Error('Project not found');

  const cacheFile = path.join(__dirname, '..', `.graph-git-${safeurl(project)}.json`);
  if (fs.existsSync(cacheFile)) return JSON.parse(fs.readFileSync(cacheFile, 'utf-8'));

  const files = glob.sync('**/*.*', {
    cwd: root,
    ignore: ['node_modules/**', '**/.*'],
    absolute: true
  });

  const nodes = [], edges = [], seen = {};
  await Promise.all(files.map(f => processFile(f, root, seen, nodes, edges)));

  const services = [...new Set(nodes.map(n => n.service))];
  const graph = { nodes, edges, services };

  fs.writeFileSync(cacheFile, JSON.stringify(graph, null, 4));
  return { nodes, edges, services: services.sort() };
}

module.exports = serveNodes;



### language file utility

match language to grammar?


In [ ]:
const fs = require('fs')
const path = require('path')
const yaml = require('js-yaml')
const mime = require('mime-types');

const YAML_PATH = path.join(__dirname, '..', 'Resources', 'Parsers', 'languages.yml')

let LANGUAGES
async function getLanguage(filepath) {
  if (!LANGUAGES) {
    LANGUAGES = (await importer.import('get antlr tool')).LANGUAGES
  }
  const fileContents = fs.readFileSync(YAML_PATH, 'utf8');
  const linguistYML = yaml.load(fileContents);

  const extName = path.extname(filepath).toLocaleLowerCase()
  const mimeType = mime.lookup(filepath)

  const linguistKeys = Object.keys(linguistYML)
  const matchedKeys = []
  for (let i = 0; i < linguistKeys.length; i++) {
    if ((linguistYML[linguistKeys[i]].extensions
      && linguistYML[linguistKeys[i]].extensions.includes(extName))
      || (linguistYML[linguistKeys[i]].filenames
        && linguistYML[linguistKeys[i]].filenames.includes(path.basename(filepath)))
      || (linguistYML[linguistKeys[i]].codemirror_mime_type
        && linguistYML[linguistKeys[i]].codemirror_mime_type == mimeType)
    ) {
      matchedKeys.push(linguistKeys[i])
    }
  }


  let matchedLanguages = []
  for (let i = 0; i < matchedKeys.length; i++) {
    if (LANGUAGES.includes(linguistYML[matchedKeys[i]].ace_mode)) {
      matchedLanguages.push(linguistYML[matchedKeys[i]].ace_mode)
    } else if (LANGUAGES.includes(linguistYML[matchedKeys[i]].codemirror_mode)) {
      matchedLanguages.push(linguistYML[matchedKeys[i]].codemirror_mode)
    } else if (LANGUAGES.includes(matchedKeys[i])) {
      matchedLanguages.push(matchedKeys[i])
    }
  }

  //console.log(matchedLanguages)

  return matchedLanguages[0]
}

module.exports = getLanguage



### repo service list

git service list?

ROUTE = /:project/service-list.json


In [ ]:
const fs = require('fs')
const path = require('path');
const glob = require('glob');

const { safeurl } = importer.import('domain cache tools')
const getLanguage = importer.import('match language to grammar')

const PROFILE_HOME = path.resolve(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE)
const GITHUB_BARE = path.join(PROFILE_HOME, 'git-bare')

async function serveServices(project) {
  const root = path.join(GITHUB_BARE, project);
  if (!fs.existsSync(root)) throw new Error('Project not found');

  const cachedPath = path.join(__dirname, '..', `.graph-git-${safeurl(project)}.json`);
  if (fs.existsSync(cachedPath)) {
    return JSON.parse(fs.readFileSync(cachedPath, 'utf-8')).services;
  }

  const files = glob.sync('**/*.*', {
    cwd: root,
    ignore: ['node_modules/**', '**/.*'],
    absolute: true,
  });

  const services = new Set();

  for (const file of files) {
    if (file.includes('cache')) continue;

    const lang = await getLanguage(file);
    if (!lang) continue;

    const service = path.relative(root, path.dirname(file));
    services.add(service);
  }

  return [...services].sort();
}

module.exports = serveServices;

